In [16]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import scipy.io
import matplotlib.pyplot as plt
import glob
import re
from IPython.display import display 
import PIL
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

class GetCars:
    
    def __init__(self, loc="car_ims/car_ims/", labels_matlab_file ="devkit/cars_annos.mat"):
        
        self.car_images_location = loc
        self.labels_mat = scipy.io.loadmat(labels_matlab_file)
        
        self.number_of_classes = len(self.labels_mat['class_names'][0])
        self.class_names = [str(self.labels_mat['class_names'][0][i][0]) for i in range(self.number_of_classes)]
        
        
        self.max_pics = 16185
        
        self.annot_df = self.create_annot_df()
        self.class_names_df = self.create_class_df()

    def bring_annot(self, fileName):
        
        if type(fileName) == str:
            numb = int(re.sub(r'[^0-9]', '', fileName))
        else:
            numb = int(fileName)

        return ((self.labels_mat['annotations'][0][numb-1]))  
    
    
    def bringup_ClassLabel(self, fileName):
        if type(fileName) == str:
            numb = int(re.sub(r'[^0-9]', '', fileName))
        else:
            numb = int(fileName)

        return self.class_names[int(self.labels_mat['annotations'][0][numb-1][5])-1]
        
        
    
    def bring_AnnotList(self, fileName):
        if type(fileName) == str:
            numb = int(re.sub(r'[^0-9]', '', fileName))
        else:
            numb = int(fileName)
        
        
        annot_array = self.bring_annot(numb)
        class_no = annot_array[5][0][0]
        subclass = search(class_no)
        annot = [numb, class_no, subclass]

        return annot  
    
    def create_annot_df(self):
        
        numb = 0
        
        annot_array = self.bring_annot(numb)
        class_no = annot_array[5][0][0]
        subclass = search(class_no)
        
        list_df = []
        
        for i in range(1, self.number_of_classes + 1):
            list_df.append((i,search(i)))
            
        cols = ["class", "subclass"]
        
        return pd.DataFrame(list_df, columns = cols)
    
    def show_annots(self, fileName):
        
        if type(fileName) == str:
            numb = int(re.sub(r'[^0-9]', '', fileName))
        else:
            numb = int(fileName)

        print("Annot:     "+str(cars.labels_mat['annotations'][0][numb-1]))
        print("File Name: "+str(cars.labels_mat['annotations'][0][numb-1][0][0]))
        print("x min:     "+str(cars.labels_mat['annotations'][0][numb-1][1][0][0]))
        print("y min:     "+str(cars.labels_mat['annotations'][0][numb-1][2][0][0]))
        print("x max:     "+str(cars.labels_mat['annotations'][0][numb-1][3][0][0]))
        print("y max:     "+str(cars.labels_mat['annotations'][0][numb-1][4][0][0]))
        print("class no:  "+str(cars.labels_mat['annotations'][0][numb-1][5][0][0]))
        print("Train(0)/Test(1): "+str(cars.labels_mat['annotations'][0][numb-1][6][0][0]))

    
    def getPath(self, number):
        fileName  = str('0') * int(6 - len(str(number)))
        fileName += str(number)+".jpg"
        return (self.car_images_location + fileName)

    
    def getImgArray(self, number, target_size=(224, 224) ):
        img_path =  self.getPath(number)
        img = image.load_img(img_path, target_size=target_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        return x
    

    def getNextImgArray(self, target_size=(224, 224) ):
        count = 0
        
        while count<self.max_pics:
            count += 1        
        
            yield (count, self.getImgArray(count, target_size))
    

    def showRandomNxN(self, N):
        plt.rcParams['figure.figsize'] = [30, 30]

        listofRand = np.random.randint(1, self.max_pics + 1, size=int(N*N))

        image_size = (128, 128)

        fig = plt.figure()

        for i in range(0, N*N):
            img = image.load_img(self.getPath(listofRand[i]), target_size=image_size)
            ax = fig.add_subplot(N, N, i+1)
            imgplot = ax.imshow(img)
            ax.set_title(self.bringup_ClassLabel(listofRand[i]))
            ax.grid(False)
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show();
    
    
    def show_classNxN(self, class_no = 20, N=4):
        plt.rcParams['figure.figsize'] = [30, 30]
        
        list_of_images = list(self.annot_df.loc[self.annot_df["class_no"] == class_no]["fileNumber"])

        listofRand = np.random.choice(list_of_images, size = N*N, replace=False)

        image_size = (128, 128)

        fig = plt.figure()

        for i in range(0, N*N):
            img = image.load_img(self.getPath(listofRand[i]), target_size=image_size)
            ax = fig.add_subplot(N, N, i+1)
            imgplot = ax.imshow(img)
            ax.set_title(self.bringup_ClassLabel(listofRand[i]))
            ax.grid(False)
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show();
        
    def show_1EachClass7x7(self, quarter = 1):
        assert quarter <= 4
        
        plt.rcParams['figure.figsize'] = [30, 30]
        
        fig = plt.figure()
        
        for class_no in range((quarter-1)*49, quarter*49):
            
            list_of_images = list(self.annot_df.loc[self.annot_df["class_no"] == class_no + 1]["fileNumber"])
            listofRand = np.random.choice(list_of_images, size = 1, replace=False)
            image_size = (128, 128)

            
            img = image.load_img(self.getPath(listofRand[0]), target_size=image_size)
            ax = fig.add_subplot(7, 7, class_no + 1 - (quarter-1)*49)
            imgplot = ax.imshow(img)
            ax.set_title("Class "+str(class_no+1)+ " " + self.bringup_ClassLabel(listofRand[0]))
            ax.grid(False)
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show();

In [17]:
# Definition der Suchkriterien im 'Type' der Images zur Vorbereitung des Dataframe
# Beispiel: Wenn 'Sedan' im Namen -> return 's'
def search(long):
    if long in car_types["Sedan"][1]:
        return 's'
        #print('s')
    elif long in car_types["Coupe"][1]:
        return 'c'
        #print('c')
    elif long in car_types["SUV"][1]:
        return 'u'
        #print('u')
    elif long in car_types["Hatchback"][1]:
        return 's'
        #print('h')
    elif long in car_types["Misc."][1]:
        return 'c'
        #print('t')
    elif long in car_types["Van"][1]:
        return 'v'
    elif long in car_types["Minivan"][1]:
        return 'v'
    elif long in car_types["Cab"][1]:
        return 't'
    elif long in car_types["Convertible"][1]:
        return 'c'
    
        #print('asdf')

In [18]:
cars = GetCars()

In [19]:
# Vorbereitung der kumulierten Gesamtliste bestehend aus den Teillisten
car_types = {
    "Sedan": [0, []],
    "Coupe": [0, []],
    "SUV": [0, []],
    "Hatchback": [0, []],
    "Convertible": [0, []],
    "Misc.": [0, []],  
    "Van": [0, []],
    "Cab": [0, []],
    "Minivan": [0, []],
}

categories = ["Sedan", "Coupe", "SUV", "Hatchback", "Convertible", "Misc.", "Van", "Cab", "Minivan"]
found_cat = [0] * cars.number_of_classes
for idx, car in enumerate(cars.class_names):
    for cat in categories:
        if cat in car:
            car_types[cat][0] += 1
            car_types[cat][1].append(idx+1)
            found_cat[idx] = 1
            break
    if found_cat[idx] ==0:
        car_types["Misc."][0] += 1
        car_types["Misc."][1].append(idx+1)
        found_cat[idx] = 1

In [20]:
print(car_types)

{'Sedan': [46, [2, 3, 5, 16, 17, 20, 23, 24, 26, 29, 35, 40, 41, 44, 49, 51, 61, 63, 67, 73, 96, 105, 115, 117, 129, 134, 135, 136, 137, 138, 140, 156, 162, 164, 165, 167, 173, 176, 177, 181, 182, 184, 185, 187, 188, 194]], 'Coupe': [27, [9, 11, 13, 14, 15, 22, 25, 28, 34, 42, 43, 46, 72, 101, 104, 112, 128, 141, 150, 151, 153, 160, 163, 171, 172, 175, 180]], 'SUV': [34, [1, 32, 33, 37, 48, 50, 52, 58, 62, 76, 89, 94, 95, 109, 110, 118, 120, 121, 131, 132, 133, 142, 143, 145, 146, 147, 148, 149, 154, 155, 159, 186, 189, 195]], 'Hatchback': [13, [7, 19, 98, 130, 139, 168, 170, 178, 183, 190, 191, 192, 193]], 'Convertible': [25, [8, 10, 12, 21, 27, 31, 36, 38, 39, 45, 55, 59, 77, 80, 81, 100, 102, 103, 107, 123, 157, 158, 161, 179, 196]], 'Misc.': [20, [4, 6, 18, 30, 47, 56, 57, 60, 66, 68, 79, 82, 83, 84, 92, 93, 97, 99, 144, 152]], 'Van': [8, [64, 71, 88, 116, 119, 166, 169, 174]], 'Cab': [18, [53, 54, 65, 69, 70, 74, 75, 86, 87, 90, 91, 106, 111, 113, 114, 122, 124, 125]], 'Minivan': 

In [21]:
# Ausgabe der PKW, die keiner genauen Kategorie zugeordnet werden können, weil 'Type' uneindeutig
# 'Misc.' wird dann zusätzlich Coupes zugeordnet, weil vorwiegend Sportwagen.
def print_car_type(car_type = "Sedan"):
    [print("Car type {} is {}".format(car, cars.class_names[car-1])) for car in car_types[car_type][1]]

print_car_type("Misc.")

Car type 4 is Acura TL Type-S 2008
Car type 6 is Acura Integra Type R 2001
Car type 18 is Audi 100 Wagon 1994
Car type 30 is BMW 3 Series Wagon 2012
Car type 47 is Buick Regal GS 2012
Car type 56 is Chevrolet Corvette ZR1 2012
Car type 57 is Chevrolet Corvette Ron Fellows Edition Z06 2007
Car type 60 is Chevrolet HHR SS 2010
Car type 66 is Chevrolet Cobalt SS 2010
Car type 68 is Chevrolet TrailBlazer SS 2009
Car type 79 is Chrysler 300 SRT-8 2010
Car type 82 is Daewoo Nubira Wagon 2002
Car type 83 is Dodge Caliber Wagon 2012
Car type 84 is Dodge Caliber Wagon 2007
Car type 92 is Dodge Magnum Wagon 2008
Car type 93 is Dodge Challenger SRT8 2011
Car type 97 is Dodge Charger SRT-8 2009
Car type 99 is FIAT 500 Abarth 2012
Car type 144 is Jaguar XK XKR 2012
Car type 152 is Lamborghini Gallardo LP 570-4 Superleggera 2012


In [22]:
#Ausgabe der Tabelle mit der jeweiligen Klasse und ihrer zugehörigen Subklasse
annots = cars.annot_df.copy()
print(annots)

     class subclass
0        1     None
1        2     None
2        3     None
3        4     None
4        5     None
..     ...      ...
191    192     None
192    193     None
193    194     None
194    195     None
195    196     None

[196 rows x 2 columns]


In [23]:
annots.to_csv("devkit/cars_label1.csv")